[![View On GitHub](https://img.shields.io/badge/View_in_Github-grey?logo=github)](https://gitlab.com/your_username/your_repository/-/blob/main/examples/showcase.ipynb)
![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)

# Cracking the Code: Practical Methods for Testing Differential Privacy

At sarus we build privacy safe analytics tools and one of our initiatives is an open source library called [qrlew](https://qrlew.readthedocs.io/en/latest/). Qrlew is used to turn sql queries into privacy-safe ones under the framework of [Differential Privacy (DP)](https://en.wikipedia.org/wiki/Differential_privacy). As the library was maturing and more features were supported we needed to test that the results generated from DP rewritten queries were actually coherent with the theory. Thanks to the approach presented here, we actually found a that in one of the mechanisms in Qrlew we were relying into wrong assumptions and corrected it straight away. This motivated us to open source our testing methodology and tools to the developers and researchers community so, let's dive in.

To build trustworthy data analysis systems, it’s important to verify that differential privacy mechanisms truly provide the privacy guarantees they promise. While there have been other attempts to create open-source libraries for differential privacy testing—such as Google’s Differential Privacy [Stochastic Tester](https://github.com/google/differential-privacy/blob/main/cc/testing/README.md), [DP Auditorium](https://github.com/google/differential-privacy/tree/main/python/dp_auditorium), and potentially others, our approach focuses on making it exceptionally easy to create datasets that resemble real-life scenarios and to effortlessly test them with any SQL query. In this ready-to-run Jupyter notebook on Google Colab, we introduce practical methods for testing differential privacy results. Adopting the perspective of an adversary attempting to breach privacy, we demonstrate how hypothesis testing can assess whether a mechanism effectively protects individual entries in a dataset.

Here, we also introduce the dp_tester repository that implements these testing techniques, showcasing how straightforward it is to apply them to your own differential privacy mechanisms. Through hands-on code examples and interactive experiments, you’ll see how easy it is to empirically validate and verify the privacy properties of your implementations using our tools.

This practical approach bridges the gap between theoretical guarantees and real-world applications, empowering developers and researchers to ensure their differential privacy solutions are both robust and effective—all within a convenient and user-friendly Colab environment.

---

## Table of Contents

- [How to test DP results](#intro)
- [Experimental settings & Results](#dp_testing_in_practice)
  1) [Dataset Generation](#dataset_generation)
  2) [Collecting DP Results](#collecting_dp_results)
  3) [Partitioning Results](#paritioning_results)
  4) [Compute Empirical Epsilons](#compute_empirical_epsilons)
- [Conclusions](#conclusions)
- [References](#references)

<a id="intro"></a>
## How to Test DP Results

Differential Privacy (DP) ensures that the output of a mechanism $\mathcal{M}$ does not significantly depend on whether any single individual is included or excluded from the dataset. Formally, a mechanism $\mathcal{M}$ is ($\epsilon,\delta$)-differentially private if:

$$\Pr[\mathcal{M}(D_0) \in S] \leq e^{\epsilon} \Pr[\mathcal{M}(D_1) \in S] + \delta$$
$\forall \; D_0$ and $D_1$, where $|D_0-D_1| \leq 1$ (neighboring datasets differing by at most one individual), $\forall \; S$ any possible output set. $\epsilon$ is the privacy loss parameter, and $\delta$ is a small failure probability.

### Adversary’s Perspective

To verify if a mechanism truly satisfies differential privacy, consider an adversary trying to determine if a particular user’s data was included. The adversary faces two situations:
- $D_0$: Dataset with the user included.
- $D_1$: Dataset with the user excluded.

The adversary’s goal is to distinguish which dataset produced the observed output. We can treat this distinction as a hypothesis testing problem:
- Null Hypothesis ($H_0$): The output Y is generated from $\mathcal{M}(D_0)$.
- Alternative Hypothesis ($H_1$): The output Y is generated from $\mathcal{M}(D_1)$.

The [Neyman–Pearson lemma](https://en.wikipedia.org/wiki/Neyman%E2%80%93Pearson_lemma) is a fundamental result in statistical hypothesis testing that provides a method for constructing the most powerful test for distinguishing between two simple hypotheses and has applications across different domains such as in medicine, physics, economy etc. The Neyman–Pearson lemma tells us that the most powerful test (one that best distinguishes $H_1$ from $H_0$) for a given error rate is based on the likelihood ratio:

$$\Lambda(Y) = \frac{P_1(Y)}{P_0(Y)}$$

where $P_0$ and $P_1$ are the probability distributions of $Y$ under $H_0$ and $H_1$, respectively. If $\Lambda(Y) \geq \tau$ for some threshold $\tau$, the test rejects $H_0$ (i.e., it guesses the dataset is $D_1$).

### Defining False Positives and False Negatives

In our scenario:
- A False Positive ($FP$) occurs if we reject $H_0$ when it is actually true. Under $H_0$, this happens when $\Lambda(Y)$ is above the threshold $\tau$:

$$ FP_{\tau} = \Pr_{H_0}[\Lambda(Y) \geq \tau] $$
- A False Negative ($FN$) occurs if we fail to reject $H_0$ when $H_1$ is true. Under $H_1$, this means $\Lambda(Y)$ is below $\tau$:
 
$$ FN_{\tau} = \Pr_{H_1}[\Lambda(Y) < \tau] $$

We can equivalently express these in terms of some region $S_{\tau}$ where $\Lambda(Y) < \tau$:

$$ FP_{\tau} = 1 - \Pr_{H_0}[Y \in S_{\tau}], \quad FN_{\tau} = \Pr_{H_1}[Y \in S_{\tau}] $$

In the graph below there is a schematic representation of our $FP$ and $FN$ definitions:

<div style="text-align: center;">
    <img src="figure_fp-fn_2.png" width="400" alt="" />
</div>

### From $FP$, $FN$ to an Empirical $\epsilon$

Since the DP definition is valid for all the output space $S$, it implies that is valid as well for a sub-region $S_{\tau}$ where:

$$ S_{\tau}= \biggl \{ \frac{\Pr \left[ \mathcal M\left( D_0 \right) \in S\right]} {\Pr\left[ \mathcal M\left(D_1\right)\in S \right]} < \tau \biggl \} = \{Y: \Lambda(Y) < \tau\} $$

Using our definitions of $FP$ and $FN$:
- $\Pr[\mathcal{M}(D_0) \in S_{\tau}] = 1 - FP_{\tau}$
- $\Pr[\mathcal{M}(D_1) \in S_{\tau}] = FN_{\tau}$

Plugging these expressions into the original definition of DP and rearranging, we get:

$$\epsilon \geq \ln\left(\frac{1 - \delta - FP_{\tau}}{FN_{\tau}}\right)  \quad \quad \forall \; D_0, D_1, |D_0-D_1| \leq 1, \quad \forall \; S_{\tau}  $$

Since this relation is valid for all possible neighboring datasets It is true as well for a set neighboring datasets and moreover it is true as well for the neighboring dataset that maximizes the empirical epsilon. Thus, it is implied that:

$$ \epsilon \geq \epsilon^{*} = \max_{\tau, \pi} \left[ ln\left(\frac {1 - \delta - FP_{\tau}} {FN_{\tau}}\right) \right] \quad \quad \forall \; (D_0, D_1) \in \{\pi_1, \pi_2, ... \pi_n\}, \pi_i = (D^i_0, D^i_1), |D^i_0-D^i_1| \leq 1, \quad \forall \; S_{\tau} $$

This relation tells us that the $\epsilon^{*}$ should never overcome the actual $\epsilon$ used by the mechanism otherwise there are potential vulnerability in the mechanism implementation. Because $FP_{\tau}$ and $FN_{\tau}$ are unknown probabilities, we estimate them empirically by running the mechanism multiple times on $D_0$ and $D_1$ and counting frequencies in different output buckets. The maximum across all tested thresholds and dataset pairs gives us:

$$ \epsilon \stackrel{?}{\geq} \hat{\epsilon^{*}} = \max_{\tau, \pi} \left[ ln\left(\frac {1 - \delta - \widehat{FP_{\tau}}} {\widehat{FN_{\tau}}}\right) \right] \quad \quad \forall \; D_0, D_1 \in \{\pi_1, \pi_2, ... \pi_n\}, \pi_i = (D^i_0-D^i_1), |D^i_0-D^i_1| \leq 1, \quad 
\forall \; S_{\tau}  $$

The $?$ indicates that this formula may not hold if $\widehat{FP_{\tau}}$ and $\widehat{FN_{\tau}}$ are poor estimators. Here, $\widehat{FP_{\tau}}$ and $\widehat{FN_{\tau}}$ are empirical estimates, we count a the number of occurrences in a specific bucket over a sufficiency large number or experiments (on the order of 10k or more). In this way the analysis presented so far is conducted for each individual bucket thus the $\hat{\epsilon^{*}}$ is the maximum across all partitions along with all neighboring datasets.


### Interpreting the Results

- Test Fails ($\hat{\epsilon}^{*} > \epsilon$): The mechanism may leak more information than allowed by $\epsilon$, indicating a potential implementation flaw.
- Test Passes ($\hat{\epsilon}^{*} \leq \epsilon$): The mechanism appears to respect the claimed differential privacy parameters for the tested dataset pairs.

Note that passing the test does not guarantee DP in all cases—only for the tested datasets and queries. However, failing the test is a strong indication that something is wrong and should be investigated further.

<a id="dp_testing_in_practice"></a>
## Experimental settings

In this section we go from theory to practice in 4 steps:
1) **Dataset Generation**: a brief description of the datasets characteristics we generated for the experiments.
2) **Collecting DP Results**: description on how we collected our DP results.
3) **Partitioning Results**: description and tools for partitioning the output space into buckets needed for computing counts.
4) **Compute Empirical Epsilons**: here we compute empirical epsilons for all partitions and compare it to the actual epsilon used for the experiments.

<a id="dataset_generation"></a>
### Dataset Generation

In the following cells we create a postgres database and we push our $D_0$ and $D_1$ datasets which differ by the removal of the data relative to 1 user.

The dataset consists in 2 tables:
- `users`: it has 100 lines each related to a distinct user. The columns are `id: (int) UNIQUE` identifying the user and `income: (float)` is a random number drawn from $\mathcal{N}(\mu=40000, \sigma=10000)$
- `transactions`: it has 10000 entries, The columns are `id: (int) UNIQUE` the transaction id, `user_id: (int)` the user who is making the transaction (it is one of the user in users), `spent: (float)` is a random number drawn from $\mathcal{U}(a=5,b=500)$, `store_id: (int)` identifier of the store, there are 200 unique stores, `other_id: (int)` is the identifier of some arbitrary feature of the user, there are 10 possible unique other ids.


In the `transactions` table the user `0` has at most 500 contributions while the remaining transactions are split uniformly among the remaining users. Moreover, the user `0` likes to make transactions in all stores and he likes them differently: the higher the store_id the higher the frequency the user `0` likes to make transactions. In other words user `0` affects all the stores in different way. The other users don't have any particular preference among the stores so the pick them randomly. Lastly, each user affects only 1 `other_id`.

$D_0$ is pushed to the default postgres schema while $D_1$ which has no information about the user with id=0 is pushed to the `D_1` schema. 

Here we will collect results from this query:

```
"SELECT store_id, SUM(spent) AS spent_per_store FROM transactions GROUP BY store_id"
```

Notice that submitting this to $D_1$ where the user affecting all `store_id`s is removed is the worst scenario.


---

In [ ]:
%%capture
# Create a postgres database
# Inspired by https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/postgresql.ipynb#scrollTo=YUj0878jPyz7
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-14
# Start postgresql server
!sudo sed -i "s/port = 5432/port = 5433/g" /etc/postgresql/14/main/postgresql.conf
!sudo service postgresql start
# Set password
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pyqrlew-db'"
%pip install git+https://github.com/sarus-tech/dp-testing.git

In [ ]:
from dp_tester.generate_datasets import generate_D_0_dataset, generate_adj_datasets
from dp_tester.constants import D_1

generate_D_0_dataset()
generate_adj_datasets(D_1, user_id=0)

<a id="collecting_dp_results"></a>
### Collecting DP Results

To collect DP results, we use the `dp_results_from_sql_query` function which uses `SqlAlchemyQueryExecutor` query executor to submit a query to the database, we use `PyqrlewDpRewriter` to compile the query with differential privacy with the provided privacy parameters
and the we use `PyqrlewTableRenamer` to change tables qualifying name to forward the query towards $D_1$.

The DP query is generated ones and is submitted many times (as much as indicated by `runs` parameter) to both $D_0$ and $D_1$.
Then results for each of the 2 dataset are stored in dictionary.

---

In [ ]:
from dp_tester.results_collector import dp_results_from_sql_query
from dp_tester.query_executors import SqlAlchemyQueryExecutor
from dp_tester.dp_rewriters import PyqrlewDpRewriter
from dp_tester.table_renamers import PyqrlewTableRenamer
from dp_tester.constants import D_0
import json

query = (
    "SELECT store_id, SUM(spent) AS spent_per_store FROM transactions GROUP BY store_id"
)
epsilon = 10.0
delta = 1e-4
runs = 10000
max_user_contributions_per_group = 10

query_executor = SqlAlchemyQueryExecutor()
dp_rewriter = PyqrlewDpRewriter(
    engine=query_executor.engine,
    max_privacy_unit_groups=max_user_contributions_per_group,
)
tables = ["users", "transactions"]
table_renamer = PyqrlewTableRenamer(dp_rewriter.dataset, tables)

results = dp_results_from_sql_query(
    non_dp_query=query,
    epsilon=epsilon,
    delta=delta,
    runs=runs,
    dp_rewriter=dp_rewriter,
    query_executor=query_executor,
    table_renamer=table_renamer,
    d_0=D_0,
    adjacent_ds=[D_1],
)

# save results
with open("results.json", "w") as outfile:
    json.dump(obj=results, fp=outfile)

<a id="paritioning_results"></a>
### Paritioning Results

Results generated by a mechanism can be described as an abstract and potentially infinite space that we refer to as $\mathcal{R}$. In order to compute $\widehat{FP}_{\tau}$ and $\widehat{FN}_{\tau}$ thus $\hat{\epsilon}^{*}$ we need to divide output space into subregions delimited by $\tau$. An effective and generic way to do so would be to define a partition function that maps the results to an integer associated with an index identifying a particular sub region: $ p: \mathcal{R} \rightarrow \mathbb{N}$

#### Buckets

In practice the result of an sql query, no matter the complexity of its structure, it can be mapped into buckets representing a sub region of the output space $\tau$. In our specific case, each result $r$ is a sequence of rows where each row is `(store_id, spent_per_store)` pair. Each `store_id` $g \in G = \{g_1, g_2, ..., g_M \}$ appears at most ones in the result since it is in the GROUP BY of the query. Buckets can be described as the disjoint set of N intervals: $I = \{ I_1, I_2, \dotsc, I_{N}\}, \quad \forall i \neq j \quad I_i \cap I_i = \emptyset, \quad I_i = [ a_{i},\ b_i ), \quad a_i, b_i, \in \mathbb{R}, \quad a_i < b_i$. Intervals are chosen such that any possible value of `spent_per_store` $x \in \mathbb{R}$ can be mapped in one of them. 

#### The partition function 
We can define a partition function as $p_g: \mathbb{R} \to \{1,2,\dots, N, \emptyset \}$, if the store g is present in the results it returns the index of the interval associated with `spent_per_store` for the store g otherwise it returns none. We have as many as there are known groups in the results as these function, we call a list of such function `partition_vector`. For tqueries like the one we are using here, such functions are provided by `QuantityOverGroups.partition_vector()`. 
The function `results_to_bucket_ids` iterates over the results and returns `t.List[t.Dict[str, t.List[int|None]]]`, Each element of the first level list is a dictionary where associated with the outcome of a particular partition function. The length of partition_vector is the same as the length of such list. Each key in the dictionary is dataset on from which we colleted the results and value is the list of integer associated to a specific bucket index. We can now compute counts to estimate $\widehat{FP}_{\tau}, \widehat{FN}_{\tau}$ and compute $\hat{\epsilon}^{*}$

In [ ]:
from dp_tester.generate_datasets import N_STORES
from dp_tester.partitioners import QuantityOverGroups
from dp_tester.analyzer import results_to_bucket_ids
from dp_tester.typing import OverallResults
import typing as t

# read results
with open("results.json") as infile:
    results = t.cast(OverallResults, json.load(infile))

NBINS = 20
partitioner = QuantityOverGroups(groups=list(range(N_STORES)))
partitioner.generate_buckets(results, n_bins=NBINS)
bucket_ids = results_to_bucket_ids(results, partitioner.partition_vector())

<a id="compute_empirical_epsilons"></a>
### Compute Empirical Epsilon

Once we have assigned each output of our mechanism $\mathcal{M}$ to a specific bucket (under both $D_0$ and $D_1$), we obtain counts: $ c_j^{D_0}$ and $c_j^{D_1}$ for each bucket $j$, where $j \in \{0, 1, \dots, n-1\}$. Let $C = \sum_{j=0}^{n-1} c_j^{D_0} = \sum_{j=0}^{n-1} c_j^{D_1}$ be the total number of runs under each scenario.

#### Sorting Buckets by Likelihood Ratio

To apply the Neyman–Pearson lemma, we consider the empirical likelihood ratio $\frac{c_j^{D_0}}{c_j^{D_1}}.$ We sort buckets in descending order so those more indicative of $D_0$ (higher value) appear first, and those more indicative of $D_1$ (lower value) appear last:

$$ \frac{c_0^{D_0}}{c_0^{D_1}} \geq \frac{c_1^{D_0}}{c_1^{D_1}} \geq \dots \geq \frac{c_{n-1}^{D_0}}{c_{n-1}^{D_1}} $$

#### Defining a Threshold $\tau_i$

Let $\tau_i$ represent a threshold set between bucket i and i+1. Intuitively:
- Buckets $ \leq i$  are more like $D_0$.
- Buckets  $> i$  are more like $D_1$.

According to the Neyman–Pearson principle, we reject $H_0$ if the data appear more indicative of $D_1$. Hence, the “reject H_0” region corresponds to buckets $j > i$.

#### Computing $FP$ and $FN$
- False Positive ($FP_{\tau_i}$): This occurs if we reject $H_0$ when $H_0$ is actually true. Under $H_0$, the probability of landing in bucket $j$ is estimated as $\frac{c_j^{D_0}}{C}$. Since rejecting $H_0$ involves buckets $j > i$:

$$ FP_{\tau_i} = \sum_{j > i} \frac{c_j^{D_0}}{C} = 1 - \sum_{j \leq i} \frac{c_j^{D_0}}{C}.$$

- False Negative ($FN_{\tau_i}$): This occurs if we fail to reject $H_0$ when $H_1$ is true. Under $H_1$, the probability of landing in bucket $j$ is $\frac{c_j^{D_1}}{C}$. Failing to reject $H_0$ corresponds to buckets $j \leq i$:

$$ FN_{\tau_i} = \sum_{j \leq i} \frac{c_j^{D_1}}{C}. $$ 


#### Computing $\hat{\epsilon}^{*}$

Recall the DP-based inequality:

$$\epsilon \geq \ln\left(\frac{1 - \delta - FP_{\tau_i}}{FN_{\tau_i}}\right)$$

We estimate $FP_{\tau_i}$ and $FN_{\tau_i}$ using the counts. Considering all $\tau_i$ and all neighboring dataset pairs $\pi$:

$$\hat{\epsilon}^{*} = \max_{\tau_i,\pi} \ln\left(\frac{1 - \delta - \widehat{FP_{\tau_i}}}{\widehat{FN_{\tau_i}}}\right)$$

We also include the symmetric case where $D_0$ and $D_1$ are swapped, ensuring both adding and removing a user are tested. This might add a second term in the $\max$ operation, where $FN$ and $FP$ are switched:

$$ \hat{\epsilon}^{*} = \max_{\tau_i,\pi} \left[ \ln\left(\frac{1 - \delta - \widehat{FP_{\tau_i}}}{\widehat{FN_{\tau_i}}}\right), \ln\left(\frac{1 - \delta - \widehat{FN_{\tau_i}}}{\widehat{FP_{\tau_i}}}\right) \right] $$


#### Adjusting the Minimum Count Threshold $T$

In practice, to reduce noise in these estimates, we often impose a minimum count threshold $T$ to ensure we only consider thresholds $\tau_i$ and subsets of the output space with enough observations. Increasing $T$ can stabilize the estimates (reducing variance in $FP$ and $FN$ computations) but may discard informative thresholds if the data is sparse. Finding a balance is key.


By following this procedure—sorting buckets, computing $FP$ and $FN$, estimating $\hat{\epsilon}^{*}$, and carefully choosing how to partition the output space and set thresholds—you can empirically validate whether a mechanism meets its differential privacy guarantees.

---

In [ ]:
from dp_tester.analyzer import empirical_epsilon
from dp_tester.analyzer import counts_from_indexes

COUNT_THRESHOLD = 5

empirical_eps_per_group = {}
for i, partition_bucket_ids in enumerate(bucket_ids):
    counts_d_0 = counts_from_indexes(
        partition_bucket_ids[D_0], len(partitioner.buckets)
    )
    counts_d_1 = counts_from_indexes(
        partition_bucket_ids[D_1], len(partitioner.buckets)
    )
    empirical_eps_per_group[i] = empirical_epsilon(
        counts_d_0,
        counts_d_1,
        delta=delta,
        counts_threshold=COUNT_THRESHOLD,
        plot=False,
    )
all_eps = list(empirical_eps_per_group.values())
max_eps = max(all_eps)

print(f"Epsilon used during the experiment: {epsilon}")
print(f"Max empirical epsilon found: {max_eps}")
print(f"Did the test passed? {max_eps <= epsilon}")

<a id="conclusions"></a>
## Conclusions

In this notebook, we demonstrated a practical method for testing differential privacy mechanisms using hypothesis testing and the Neyman–Pearson lemma. By leveraging the dp_testing library, we showed how to generate realistic datasets, apply differential privacy, and empirically evaluate the resulting privacy guarantees. This approach not only helped us detect and correct a flaw in one of our DP implementations, but it also provides a general framework that developers and researchers can use to ensure their differential privacy solutions are both robust and reliable.


<a id="references"></a>
## References

- [Differencial Privacy](https://maxkasy.github.io/home/files/other/ML_Econ_Oxford/differential_privacy.pdf)
- [Wilson et al.](https://arxiv.org/abs/1909.01917)
- [Kairouz et al.](https://arxiv.org/abs/1311.0776)
- [Milad Nasr et al.](https://arxiv.org/abs/2101.04535)